<center> <h1 style="background-color:seagreen; color:white" >Agrupamento Hierárquico com HierarchicalBisecting com Spark usando PySpark</h1></center>  

> + **Aglomerativa**: começa com pontos em grupos individuais e a cada etapa funde os pares mais próximos. Requer uma noção de proximidade. Mais comuns. 
> + **Divisiva**: Começa incluindo todos, e a cada etapa divide até que reste apenas grupos únicos

<img src="diagrama.png" alt="Cluster Hierárquico" width="500" height="300">

<center> <h2 style="background-color:DarkKhaki; color:white" >Inicializando o PySpark</h2></center> 

In [2]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("HierarchicalBisecting").getOrCreate()

<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Hiper Parâmetros</h3></center> 


> + **distanceMeasure**: Medida de distância: Opções: “euclidean” e “cosine”. (padrão: euclidean)
> + **k**: Número de clusters. (padrão: 4)
> + **maxIter**: número máximo de iteração. (padrão: 20)
> + **minDivisibleClusterSize**: o número minimo de divisões se >=1 ou a proporção minima se <1 (padrão: 1)




---

<center> <h1 style="background-color:seagreen; color:white" >Aplicando Agrupamento Hierárquico com HierarchicalBisecting  no DataSet Íris</h1></center>



<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Carregando o Cunjunto de Dados Íris</h3></center> 

In [7]:
iris = spark.read.csv("../Material_do_Curso/iris.csv",
                             header=True, inferSchema=True, sep=",")
print(f"Quantidade de Registros do Dataset: {iris.count()}")
iris.show(5, truncate=True)

Quantidade de Registros do Dataset: 150
+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



<center> <h2 style="background-color:DarkKhaki; color:white" >Escolha das Variáveis Independentes e Dependente para treinamento do Modelo</h2></center> 


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Importação do Módulo do PySpark Para o Pré-Processamento dos Dados</h3></center> 

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
# Colocando os dados no Formato de dataframe do Spark usando o VectorAssembler
asb = VectorAssembler(inputCols=["sepallength", "sepalwidth",
                                      "petallength", "petalwidth"],
                      outputCol="features")
iris_rf = asb.transform(iris)
iris_rf.show(5, truncate=False)

+-----------+----------+-----------+----------+-----------+-----------------+
|sepallength|sepalwidth|petallength|petalwidth|class      |features         |
+-----------+----------+-----------+----------+-----------+-----------------+
|5.1        |3.5       |1.4        |0.2       |Iris-setosa|[5.1,3.5,1.4,0.2]|
|4.9        |3.0       |1.4        |0.2       |Iris-setosa|[4.9,3.0,1.4,0.2]|
|4.7        |3.2       |1.3        |0.2       |Iris-setosa|[4.7,3.2,1.3,0.2]|
|4.6        |3.1       |1.5        |0.2       |Iris-setosa|[4.6,3.1,1.5,0.2]|
|5.0        |3.6       |1.4        |0.2       |Iris-setosa|[5.0,3.6,1.4,0.2]|
+-----------+----------+-----------+----------+-----------+-----------------+
only showing top 5 rows



In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
idx = StringIndexer(inputCol="class", outputCol="grupos")
iris_rfsi = idx.fit(iris_rf).transform(iris_rf)
iris_rfsi.show(5, truncate=False)

+-----------+----------+-----------+----------+-----------+-----------------+------+
|sepallength|sepalwidth|petallength|petalwidth|class      |features         |grupos|
+-----------+----------+-----------+----------+-----------+-----------------+------+
|5.1        |3.5       |1.4        |0.2       |Iris-setosa|[5.1,3.5,1.4,0.2]|0.0   |
|4.9        |3.0       |1.4        |0.2       |Iris-setosa|[4.9,3.0,1.4,0.2]|0.0   |
|4.7        |3.2       |1.3        |0.2       |Iris-setosa|[4.7,3.2,1.3,0.2]|0.0   |
|4.6        |3.1       |1.5        |0.2       |Iris-setosa|[4.6,3.1,1.5,0.2]|0.0   |
|5.0        |3.6       |1.4        |0.2       |Iris-setosa|[5.0,3.6,1.4,0.2]|0.0   |
+-----------+----------+-----------+----------+-----------+-----------------+------+
only showing top 5 rows



<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Convertendo os valores float da coluna grupos para inteiro</h3></center> 

In [14]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

iris_rfsi = iris_rfsi.withColumn("grupos", iris_rfsi["grupos"].cast(IntegerType()))
iris_rfsi.show(5)

+-----------+----------+-----------+----------+-----------+-----------------+------+
|sepallength|sepalwidth|petallength|petalwidth|      class|         features|grupos|
+-----------+----------+-----------+----------+-----------+-----------------+------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|     0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|     0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|     0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|     0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|     0|
+-----------+----------+-----------+----------+-----------+-----------------+------+
only showing top 5 rows



---


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" > Importação do Módulo do PySpark Para Criação do Modelo de Agrupamentos com Kmeans</h3></center> 


In [15]:
from pyspark.ml.clustering import BisectingKMeans

#### Instanciando Objeto e criando o Modelo

In [16]:
# Instanciando o objeto BisectingKMeans
obj_bkm = BisectingKMeans(featuresCol="features", predictionCol="pre_grupo", maxIter=100, k=3)

# Criando o Modelo
model_bkm = obj_bkm.fit(iris_rfsi)

In [17]:
grupos = model_bkm.transform(iris_rfsi)

In [18]:
grupos.show(150)

+-----------+----------+-----------+----------+---------------+-----------------+------+---------+
|sepallength|sepalwidth|petallength|petalwidth|          class|         features|grupos|pre_grupo|
+-----------+----------+-----------+----------+---------------+-----------------+------+---------+
|        5.1|       3.5|        1.4|       0.2|    Iris-setosa|[5.1,3.5,1.4,0.2]|     0|        0|
|        4.9|       3.0|        1.4|       0.2|    Iris-setosa|[4.9,3.0,1.4,0.2]|     0|        0|
|        4.7|       3.2|        1.3|       0.2|    Iris-setosa|[4.7,3.2,1.3,0.2]|     0|        0|
|        4.6|       3.1|        1.5|       0.2|    Iris-setosa|[4.6,3.1,1.5,0.2]|     0|        0|
|        5.0|       3.6|        1.4|       0.2|    Iris-setosa|[5.0,3.6,1.4,0.2]|     0|        0|
|        5.4|       3.9|        1.7|       0.4|    Iris-setosa|[5.4,3.9,1.7,0.4]|     0|        0|
|        4.6|       3.4|        1.4|       0.3|    Iris-setosa|[4.6,3.4,1.4,0.3]|     0|        0|
|        5

---

<center> <h1 style="background-color:seagreen; color:white" >Avaliando o Modelo</h1></center> 
<br>



<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Avaliação usando a Confusion Matrix e Accuracy Score</h3></center> 

In [20]:
classe = grupos.select("grupos").collect()
agrupado = grupos.select("pre_grupo").collect()

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(classe, agrupado)
print(cm)

[[50  0  0]
 [ 3 45  2]
 [ 0 14 36]]


In [24]:
acuracia = accuracy_score(classe, agrupado)
print(f"Acurácia: {acuracia}")

Acurácia: 0.8733333333333333


In [22]:
acuracia = (cm[0, 0] + cm[1, 1] + cm[2, 2])/150
print(f"Acurácia: {acuracia}")

Acurácia: 0.8733333333333333


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Avaliação usando a Confusion Matrix e Accuracy Score</h3></center> 


> + Coeficiente Silhouette
> + Medida do quanto cada ponto em um cluster está próximo do cluster vizinho
> + Valor entre -1 e 1
> + Próximo a 0 indica que os pontos estão próximos do limite
> + Próximo a 1 indica que os pontos estão distantes do cluster vizinho
> + Próximo a -1 indica que os pontos estão no cluster errado

In [25]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [26]:
metrica = ClusteringEvaluator(predictionCol="grupos")
silhouette = metrica.evaluate(grupos)

print(f"silhouette: {silhouette}")

silhouette: 0.65646792310419


A métrica ***silhouette*** mostra que o modelo conseguir agrupar corretamente as classes, já que seu valor está mais próximo de 1.